In [ ]:
import pandas as pd
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [ ]:
from scripts.data_loader import load_data

In [ ]:
file_path = r"C:\Users\fikad\Desktop\10acedamy\Week_1\Data\raw_analyst_ratings.csv"

# Load the dataset
data = load_data(file_path)